## **1. Import libraries**

In [ ]:
import math
import os
import re
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

## **2. Download and load dataset**

In [ ]:
# https://drive.google.com/file/d/1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc/view?usp=sharing
!gdown --id 1KfrBAycsgQBt1mtEbzJh5pSYL8YIk0Tc
!unzip poem_dataset_final.zip

In [ ]:
DATASET_PATH = 'datasets/poem_final.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [ ]:
df['content'][0].split('\n')

['Cái làm ta hạnh phúc',
 'Thực ra cũng chẳng nhiều',
 'Chỉ cần có ai đó',
 'Để ta thầm thương yêu',
 '',
 'Rồi thêm chút công việc',
 'Cho ta làm hàng ngày',
 'Cuối cùng, chút mơ mộng',
 'Để đưa ta lên mây']

## **3. Build vectorization function**

In [ ]:
def text_normalize(text):
    text = text.strip()

    return text

df['content'] = df['content'].apply(lambda x: text_normalize(x))

In [ ]:
def tokenizer(text):
    return text.split()

def yield_tokens(df):
    for idx, row in df.iterrows():
        yield tokenizer(row['content'])

vocab = build_vocab_from_iterator(
    yield_tokens(df),
    specials=['<unk>', '<pad>', '<sos>', '<eos>', '<eol>']
)
vocab.set_default_index(vocab['<unk>'])
vocab.get_stoi()

{'“Liệu': 2198,
 'ẩm': 2196,
 'ầm': 2195,
 'đợi,': 2193,
 'địa': 2187,
 'đền!': 2186,
 'đếm': 2185,
 'đặn': 2184,
 'đáng': 2182,
 'điên': 2178,
 'Được': 2174,
 'Đâu': 2172,
 'Đàn': 2171,
 'ô-kê': 2169,
 'Ðức': 2164,
 'Ðêm': 2162,
 'Ðã': 2161,
 'xấu': 2158,
 'xúc': 2156,
 'xoãi': 2154,
 'vụng': 2151,
 'về…': 2150,
 'vạng': 2148,
 'tột': 2144,
 'tổng': 2143,
 'tất': 2141,
 'túi': 2138,
 'tôi.': 2137,
 'tình.': 2134,
 'trưởng': 2132,
 'trách': 2129,
 'trung,': 2128,
 'trao': 2127,
 'toát': 2126,
 'thỏ': 2121,
 'thong': 2114,
 'xiêu': 2153,
 'thiểu': 2112,
 'sướng': 2110,
 'rợ.': 2104,
 'rộng': 2103,
 'rẩy': 2102,
 'rèn': 2100,
 'riết': 2096,
 'phứt': 2093,
 'phú': 2091,
 'phù.': 2090,
 'phím': 2089,
 'nứt': 2083,
 'nợ': 2081,
 'thon': 2113,
 'nề': 2079,
 'nước...': 2078,
 'vang': 2145,
 'nông': 2076,
 'nó,': 2075,
 'nâu': 2074,
 'nuốt': 2073,
 'nhằn': 2070,
 'nhạt': 2067,
 'nhưng': 2066,
 'nhung': 2065,
 'nhiệm': 2063,
 'ngó': 2058,
 'nghị': 2056,
 'mục.': 2054,
 'mổ': 2053,
 'mệt': 2051,

In [ ]:
PAD_TOKEN = vocab['<pad>']
EOS_TOKEN = vocab['<eos>']

MAX_SEQ_LEN = 25

def pad_and_truncate(input_ids, max_seq_len):
    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
    else:
        input_ids += [PAD_TOKEN] * (max_seq_len - len(input_ids))

    return input_ids

def vectorize(text, max_seq_len):
    input_ids = [vocab[token] for token in tokenizer(text)]
    input_ids = pad_and_truncate(input_ids, max_seq_len)

    return input_ids

def decode(input_ids):
    return [vocab.get_itos()[token_id] for token_id in input_ids]

In [ ]:
vocab.get_itos()[0]

'<unk>'

In [ ]:
print(df['content'][0].split('\n')[0])
print(vectorize(df['content'][0].split('\n')[0], 10))

Cái làm ta hạnh phúc
[175, 62, 39, 313, 366, 1, 1, 1, 1, 1]


## **4. Create Poem Dataset**

In [ ]:
class PoemDataset(Dataset):
    def __init__(self, df, tokenizer, vectorizer, max_seq_len):
        self.tokenizer = tokenizer
        self.vectorizer = vectorizer
        self.max_seq_len = max_seq_len
        self.input_seqs, self.target_seqs, self.padding_masks = self.create_samples(df)

    def create_padding_mask(self, input_ids, pad_token_id=PAD_TOKEN):
        return [0 if token_id == pad_token_id else 1 for token_id in input_ids]

    def split_content(self, content):
        samples = []

        poem_parts = content.split('\n\n')
        for poem_part in poem_parts:
            poem_in_lines = poem_part.split('\n')
            if len(poem_in_lines) == 4:
                samples.append(poem_in_lines)

        return samples

    def prepare_sample(self, sample):
        input_seqs = []
        target_seqs = []
        padding_masks = []

        input_text = '<sos> ' + ' <eol> '.join(sample) + ' <eol> <eos>'
        input_ids = self.tokenizer(input_text)
        for idx in range(1, len(input_ids)):
            input_seq = ' '.join(input_ids[:idx])
            target_seq = ' '.join(input_ids[1:idx+1])
            input_seq = self.vectorizer(input_seq, self.max_seq_len)
            target_seq = self.vectorizer(target_seq, self.max_seq_len)
            padding_mask = self.create_padding_mask(input_seq)

            input_seqs.append(input_seq)
            target_seqs.append(target_seq)
            padding_masks.append(padding_mask)

        return input_seqs, target_seqs, padding_masks

    def create_samples(self, df):
        input_seqs = []
        target_words = []
        padding_masks = []

        for idx, row in df.iterrows():
            content = row['content']
            samples = self.split_content(content)
            for sample in samples:
                sample_input_seqs, sample_target_words, sample_padding_masks = self.prepare_sample(sample)

                input_seqs += sample_input_seqs
                target_words += sample_target_words
                padding_masks += sample_padding_masks

        input_seqs = torch.tensor(input_seqs, dtype=torch.long)
        target_words = torch.tensor(target_words, dtype=torch.long)
        padding_masks = torch.tensor(padding_masks, dtype=torch.float)

        return input_seqs, target_words, padding_masks

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        input_seqs = self.input_seqs[idx]
        target_seqs = self.target_seqs[idx]
        padding_masks = self.padding_masks[idx]

        return input_seqs, target_seqs, padding_masks

TRAIN_BS = 256
train_dataset = PoemDataset(
    df=df,
    tokenizer=tokenizer,
    vectorizer=vectorize,
    max_seq_len=MAX_SEQ_LEN
)
train_loader = DataLoader(
    train_dataset,
    batch_size=TRAIN_BS,
    shuffle=False
)

In [ ]:
input_seqs, target_seqs, padding_masks = next(iter(train_loader))

print(input_seqs[0])
print(target_seqs[0])
print(padding_masks[0])

tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1])
tensor([175,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])


In [ ]:
for idx in range(MAX_SEQ_LEN):
    print(decode(input_seqs[idx]))
    print(decode(target_seqs[idx]))

['<sos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['Cái', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'Cái', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['Cái', 'làm', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'Cái', 'làm', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

['<sos>', 'Cái', 'làm', 'ta', 'hạnh', 'phúc', '<eol>', 'Thực', 'ra', 'cũng', 'chẳng', 'nhiều', '<eol>', 'Chỉ', 'cần', 'có', 'ai', 'đó', '<eol>', 'Để', 'ta', 'thầm', 'thương', 'yêu', '<pad>']
['Cái', 'làm', 'ta', 'hạnh', 'phúc', '<eol>', 'Thực', 'ra', 'cũng', 'chẳng', 'nhiều', '<eol>', 'Chỉ', 'cần', 'có', 'ai', 'đó', '<eol>', 'Để', 'ta', 'thầm', 'thương', 'yêu', '<eol>', '<pad>']
['<sos>', 'Cái', 'làm', 'ta', 'hạnh', 'phúc', '<eol>', 'Thực', 'ra', 'cũng', 'chẳng', 'nhiều', '<eol>', 'Chỉ', 'cần', 'có', 'ai', 'đó', '<eol>', 'Để', 'ta', 'thầm', 'thương', 'yêu', '<eol>']
['Cái', 'làm', 'ta', 'hạnh', 'phúc', '<eol>', 'Thực', 'ra', 'cũng', 'chẳng', 'nhiều', '<eol>', 'Chỉ', 'cần', 'có', 'ai', 'đó', '<eol>', 'Để', 'ta', 'thầm', 'thương', 'yêu', '<eol>', '<eos>']


## **5. Create model**

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dims, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dims, 2) * (-math.log(10000.0) / embedding_dims))
        pe = torch.zeros(max_len, 1, embedding_dims)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)

        return x

class TransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dims,
        n_heads,
        hidden_dims,
        n_layers,
        dropout=0.5
    ):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.embedding = nn.Embedding(vocab_size, embedding_dims)
        self.embedding_dims = embedding_dims

        self.pos_encoder = PositionalEncoding(embedding_dims, dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            embedding_dims,
            n_heads,
            hidden_dims,
            dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.linear = nn.Linear(embedding_dims, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask=None, padding_mask=None):
        src = self.embedding(src) * math.sqrt(self.embedding_dims)
        src = self.pos_encoder(src)
        if src_mask is None:
            src_mask = nn.Transformer.generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=padding_mask)
        output = self.linear(output)

        return output

In [ ]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMS = 128
HIDDEN_DIMS = 128
N_LAYERS = 2
N_HEADS = 4
DROPOUT = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_tests = torch.randint(1, 10, (1, 10)).to(device)

model = TransformerModel(
    VOCAB_SIZE,
    EMBEDDING_DIMS,
    N_HEADS,
    HIDDEN_DIMS,
    N_LAYERS,
    DROPOUT
).to(device)

with torch.no_grad():
    output = model(input_tests)
    print(output.shape)

## **6. Training**

In [ ]:
LR = 5.0
EPOCHS = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

In [ ]:
model.train()
for epoch in range(EPOCHS):
    losses = []
    for idx, samples in enumerate(train_loader):
        input_seqs, target_seqs, padding_masks = samples
        input_seqs = input_seqs.to(device)
        target_seqs = target_seqs.to(device)
        padding_masks = padding_masks.to(device).permute(1, 0)

        output = model(input_seqs, padding_mask=padding_masks)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target_seqs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f'EPOCH {epoch+1}\tLoss {total_loss}')
    scheduler.step()

EPOCH 1	Loss 4.639690832658247
EPOCH 2	Loss 3.625998681241816
EPOCH 3	Loss 3.2110321792689236
EPOCH 4	Loss 2.788922759619626
EPOCH 5	Loss 2.283224804834886
EPOCH 6	Loss 1.8168869641694156
EPOCH 7	Loss 1.4792465662414378
EPOCH 8	Loss 1.2354302108287811
EPOCH 9	Loss 1.0571322508833625
EPOCH 10	Loss 0.9457327005538073
EPOCH 11	Loss 0.8403109799731862
EPOCH 12	Loss 0.7786144885149869
EPOCH 13	Loss 0.7101930718530308
EPOCH 14	Loss 0.691323733465238
EPOCH 15	Loss 0.6552018868652257
EPOCH 16	Loss 0.6382090970873833
EPOCH 17	Loss 0.6235277822071855
EPOCH 18	Loss 0.6107619580897418
EPOCH 19	Loss 0.5927509387785738
EPOCH 20	Loss 0.5788861607963388
EPOCH 21	Loss 0.5693113173950802
EPOCH 22	Loss 0.5591132322495634
EPOCH 23	Loss 0.5465741496194493
EPOCH 24	Loss 0.5366708399219946
EPOCH 25	Loss 0.528262811628255
EPOCH 26	Loss 0.5211723704229702
EPOCH 27	Loss 0.5142657607793808
EPOCH 28	Loss 0.5065084004944022
EPOCH 29	Loss 0.5007355809211731
EPOCH 30	Loss 0.4971333308653398
EPOCH 31	Loss 0.487942457

## **7. Inference**

In [ ]:
def sample_with_temperature(logits, temperature=1.0):
    if temperature != 1.0:
        logits = logits / temperature

    probabilities = F.softmax(logits, dim=-1)

    sampled_index = torch.multinomial(probabilities, 1).item()

    return sampled_index

In [ ]:
model.eval()
temperature = 1.2
input_text = '<sos> Anh'
input_tokens = tokenizer(input_text)
input_ids = [vocab[token] for token in input_tokens]
eos_token_id = vocab['<eos>']
generated_ids = input_ids.copy()
MAX_GENERATION_LEN = 50
for _ in range(MAX_GENERATION_LEN):
    input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)

    last_token_logits = outputs[0, -1, :]
    next_token_id = sample_with_temperature(last_token_logits, temperature)
    generated_ids.append(next_token_id)

    if next_token_id == eos_token_id:
        break

# Convert the generated tokens back to text
generated_text = decode(generated_ids)
generated_text = ' '.join(generated_text)
generated_text = generated_text.replace('<sos>', '')
lines = generated_text.split('<eol>')
for line in lines:
    print(''.join(line))

 Anh đi qua đời em những đêm dài 
 Khù khờ mà bướng bỉnh 
 Vượt trở ngại là không ở hang sâu 
 Một thoáng một thời ngây thơ 
 Như ta còn đâu 
 Quanh xác thuyền sóng giạt 
 ai cười mà ầm lên thay cơm 
 Nên
